In [1]:
import numpy as np
from numpy import dtype
import netCDF4 as nc
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import sklearn
from scipy import stats,signal
from scipy.spatial.distance import cdist
import datetime as dt
from datetime import date
import cartopy.crs as ccrs
import cartopy.feature as cf
import cartopy.mpl.ticker as cticker
import cftime
import dask
import collections
import scipy as scipy
from scipy.signal import detrend
from scipy import stats,signal
import cfgrib
import os
import glob
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_score
from eofs.standard import Eof
from scipy.signal import butter, filtfilt

In [ ]:
#define the latlon domain and the time period
lat1=24;lat2=55
lon1=230;lon2=300
lev0 = 500
iyr1=1960; iyr2=2022
nyr=iyr2-iyr1+1
dt01='-04-01';dt02='-07-31'
ndy=(date(1979,7,31)-date(1979,4,1)).days +1

In [ ]:
#number of weather regimes
nclusters=5

In [ ]:
#input file info (ERA5 is publically available)
dir = '/glade/u/home/mgraber/ERA5/annualdata/*.nc'
df = xr.open_mfdataset(dir)

In [ ]:
# Calculate geopotential Height at 500 hPa
heights = df.z/9.80665
df['heights'] = heights

In [ ]:
df.coords['longitude'] = (df.coords['longitude'] + 360)
df = df.sortby(df.longitude)

In [ ]:
#read daily z500 data
#note that the latitude dimension is reversed
for iyr in range(iyr1,iyr2+1):
   tm1=str(iyr)+dt01 
   tm2=str(iyr)+dt02 
   hgt0=df.heights.sel(time=slice(tm1,tm2), longitude=slice(lon1,lon2),latitude=slice(lat2,lat1))
   if iyr==iyr1:	#initialize hgt and extract latlon info
      lats=hgt0['latitude'].values;
      lons=hgt0['longitude'].values;
      nn,nlat,nlon=hgt0.shape
      heights=np.zeros((nyr,ndy,nlat,nlon))
   hgt01=xr.concat([hgt0], dim='time')
   heights[iyr-iyr1,:,:,:]=hgt01.rolling(time=5,center=True,min_periods=1).mean()
print('Finish reading H500')
hgt2=heights[0]

In [ ]:
# Input here annual means of the northern hemisphere (1960-2023)
means = np.array([5639.813, 5632.539, 5631.536, 5630.46, 5621.321, 5629.218, 5631.369, 5628.906, 5634.269, 5632.497, 5630.712, 5633.057, 5629.118, 5635.877, 5622.955, 5633.181, 5622.389, 5634, 5634.116, 5640.860, 5642.944, 5639.301, 5635.26, 5636.912, 5634.96, 5635.571, 5633.023, 5643.526, 5647.316, 5637.255, 5636.718, 5642.672, 5626.458, 5645.321, 5635.684, 5648.243, 5636.616, 5640.313, 5656, 5641.954, 5636.824, 5636.632, 5639.409, 5643.032, 5649.548, 5651.986, 5646.229, 5645.056, 5646.278, 5641.797, 5658.013, 5646.117, 5651.6, 5645.093, 5643.743, 5650.469, 5663.132, 5646.585, 5648.475, 5665.287, 5663.693, 5653.840, 5652.557, 5656.943])
means_2 = means[0:63]

In [ ]:
# subtract seasonal mean and detrend
heights = heights-means_2[:,None,None,None]
heights = signal.detrend(heights,axis=0)

In [ ]:
#calculate a cosine weighting function
lon2d,lat2d=np.meshgrid(lons,lats)
wgt=np.cos(lat2d*np.pi/180.0)
wgts=np.repeat(wgt[np.newaxis,:],nyr*ndy,axis=0)

In [ ]:
#reshape the data to [ny*nday, nlon*nlat] to make it suitable for K-means
hgt0=np.reshape(heights,(nyr*ndy,nlat,nlon))	#un-weighted H500
wgts=np.reshape(wgts,(nyr*ndy,nlat*nlon))
heights=np.reshape(heights,(nyr*ndy,nlat*nlon))
heights=heights*wgts

In [ ]:
#apply the k-mean cluster analysis
km_standard = KMeans(n_clusters=nclusters,random_state=0).fit(heights)
labels = km_standard.labels_ 		#of the dimension (nyr*ndy)
WR0 = km_standard.cluster_centers_	#cluster centers
WR0 = np.reshape(WR0,(nclusters,nlat,nlon))

In [ ]:
# Calculate the seasonal frequency
# Calculate the cluster freq.

length = len(labels)
years=[]
seascounts = []
for x in range(int(length/122)):  # 122 represents the number of days per year in the warm season
        years.append(labels[x*122:(x+1)*122])
    
        seascounts.append(np.unique(years[x],return_counts=True))

unique,counts=np.unique(labels, return_counts=True)
freq=np.round(100.0*counts/len(labels),1)

In [ ]:
#calculate the cluster mean using un-weighted data
WRs=np.zeros((nclusters,nlat,nlon)) 
for i in unique:
    D=np.where(labels==i)
    WRs[i]=np.nanmean(hgt0[D],axis=0)

In [ ]:
#order WRs with descending freq. by multiplying -1.0
ii=np.argsort(-1.0*freq)
freq=freq[ii]
WRs=WRs[ii]